<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/DAMPFER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
life_time_final = 60 * 60 * 24 * 30 * 12 * 50
time_step = 300
Pinitial = 2000
Pambient = 1000
V = 0.001
Pcritic = 1500
Qinitial_vals = np.arange(1.1E-9, 1E-8, 1e-9)
time_actual = np.arange(time_step, life_time_final, time_step)

df_list = []

for Qinitial in Qinitial_vals[:1]:
    # Defining starting variables
    Premain_start = Pinitial - (Qinitial * time_step / V)
    Q_start = (Qinitial * ((Premain_start**2) - (Pambient**2)) / ((Pinitial**2) - (Pambient**2)))
    Pambient_start = Pambient+Pinitial-Premain_start

    # Premain_list = []
    # Premain_list.append(Pinitial)
    # Premain_list.append(Premain_start)

    # Pambient_list = []
    # Pambient_list.append(Pambient)
    # Pambient_list.append(Pambient_start)

    # Q_calc_future_list = []
    # Q_calc_future_list.append(Qinitial)
    # Q_calc_future_list.append(Q_start)

    # Premain_calc_buffer = Premain_start
    # Premain_calc_buffer_before = Premain_start
    # Pambient_calc_buffer = Pambient_start

    # Q_buffer_calc = Q_start

    # i = 0
    # while Premain_calc_buffer > Pcritic and Pambient_calc_buffer < Pcritic:
    #     Premain_calc_buffer = Premain_calc_buffer - (Q_buffer_calc * time_step / V)
    #     Premain_list.append(Premain_calc_buffer)

    #     Pambient_calc_buffer = Pambient + (Pinitial-Premain_calc_buffer)
    #     Pambient_list.append(Pambient_calc_buffer)

    #     calc = Q_buffer_calc * ((Premain_calc_buffer**2) - (Pambient**2)) / ((Premain_calc_buffer_before**2) - (Pambient**2))
    #     Q_calc_future_list.append(calc)
    #     Q_buffer_calc = calc

    #     Premain_calc_buffer_before = Premain_calc_buffer

    #     i+=1


    Premain_real_list = []
    Premain_real_list.append(Pinitial)
    Premain_real_list.append(Premain_start)

    Pambient_real_list = []
    Pambient_real_list.append(Pambient)
    Pambient_real_list.append(Pambient_start)

    Q_real_future_list = []
    Q_real_future_list.append(Qinitial)
    Q_real_future_list.append(Q_start)

    Premain_real_buffer = Premain_start
    Premain_real_buffer_before = Premain_start
    Pambient_real_buffer = Pambient_start
    Pambient_real_buffer_before = Pambient_start

    Q_buffer_real = Q_start

    i = 0
    while Premain_real_buffer > Pcritic and Pambient_real_buffer < Pcritic :
        Premain_real_buffer = Premain_real_buffer - (Q_buffer_real * time_step / V)
        Premain_real_list.append(Premain_real_buffer)

        Pambient_real_buffer = Pambient + (Pinitial-Premain_real_buffer)
        Pambient_real_list.append(Pambient_real_buffer)

        calc = Q_buffer_real * ((Premain_real_buffer**2) - (Pambient_real_buffer_before**2)) / ((Premain_real_buffer_before**2) - (Pambient_real_buffer**2))
        Q_real_future_list.append(calc)
        Q_buffer_real = calc

        Premain_real_buffer_before = Premain_real_buffer
        Pambient_real_buffer_before = Pambient_real_buffer

        i+=1

    # Q_diff = np.array(Q_real_future_list)-np.array(Q_calc_future_list)

    # df = pd.DataFrame({'Qcalc': Q_calc_future_list, 'Qreal': Q_real_future_list, 'Qdiff': Q_diff, 'Pinternal': Premain_list, 'Penv': Pambient_list, 'Duration': time_actual[:len(Premain_list)]})

    # #Converting duration in seconds to specified time variables
    # df['Duration_min']   = df['Duration']/60
    # df['Duration_hour']  = df['Duration_min']/60
    # df['Duration_day']   = df['Duration_hour']/24
    # df['Duration_month'] = df['Duration_day']/30
    # df['Duration_year']  = df['Duration_month']/12

    # #Adding all the df's that depends different Qinitial values to a list
    # df_list.append(df)

In [40]:
# pd.options.display.float_format = '{:.25e}'.format

In [ ]:
data = {
    "Qcalc": Q_calc_future_list,
    "Penv_calc": Pambient_list,
    "Pint_calc": Premain_list,
    "Duration": time_actual[:len(Premain_list)]
}

data['Duration_min']   = data['Duration']/60
data['Duration_hour']  = data['Duration_min']/60
data['Duration_day']   = data['Duration_hour']/24
data['Duration_month'] = data['Duration_day']/30
data['Duration_year']  = data['Duration_month']/12

data = pd.DataFrame(data)
data.tail(10)

In [4]:
data = {
    "Qreal": Q_real_future_list,
    "Penv_real": Pambient_real_list,
    "Pint_real": Premain_real_list,
    "Duration": time_actual[:len(Premain_real_list)]
}

data = pd.DataFrame(data)

data['Duration_min']   = data['Duration']/60
data['Duration_hour']  = data['Duration_min']/60
data['Duration_day']   = data['Duration_hour']/24
data['Duration_month'] = data['Duration_day']/30
data['Duration_year']  = data['Duration_month']/12

data.head()

ValueError: ignored

In [5]:
Q_real_future_list

[1.1e-09,
 1.0999995160000398e-09,
 1.0999987900001995e-09,
 1.0999980640008385e-09,
 1.0999973380019567e-09,
 1.0999966120035542e-09,
 1.0999958860056307e-09,
 1.099995160008186e-09,
 1.0999944340112206e-09,
 1.0999937080147345e-09,
 1.0999929820187273e-09,
 1.0999922560231995e-09,
 1.099991530028151e-09,
 1.0999908040335815e-09,
 1.0999900780394914e-09,
 1.0999893520458805e-09,
 1.0999886260527482e-09,
 1.099987900060095e-09,
 1.0999871740679215e-09,
 1.099986448076227e-09,
 1.0999857220850115e-09,
 1.0999849960942751e-09,
 1.0999842701040181e-09,
 1.0999835441142402e-09,
 1.0999828181249413e-09,
 1.0999820921361213e-09,
 1.0999813661477807e-09,
 1.0999806401599192e-09,
 1.0999799141725368e-09,
 1.099979188185634e-09,
 1.09997846219921e-09,
 1.0999777362132654e-09,
 1.0999770102277996e-09,
 1.099976284242813e-09,
 1.0999755582583055e-09,
 1.0999748322742771e-09,
 1.0999741062907281e-09,
 1.0999733803076585e-09,
 1.0999726543250678e-09,
 1.0999719283429566e-09,
 1.099971202361324e-09,